In [1]:
# Declare Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [2]:
# Choose the executable path to driver 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# HTML Object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve the latest element that contains news title and news_paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

# Display scrapped data 
print(news_title)
print(news_p)

After a Reset, Curiosity Is Operating Normally
NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. 


### JPL Mars Space Images - Featured Image

In [5]:
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_img)

In [6]:
#HTML Object
html_img = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_img, 'html.parser')
base_img_url = soup.find('article', class_='carousel_item')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
# print(base_img_url)
featured_img_url = f'https://www.jpl.nasa.gov{base_img_url}'
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18899-1920x1200.jpg


### Mars Weather

In [7]:
# Visit the Mars Weather twitter account
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [8]:
html_weather = browser.html
soup = BeautifulSoup(html_weather, "html.parser")
#temp = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
# Find all elements that contain tweets
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')

# Retrieve all elements that contain news title in the specified range
# Look for entries that display weather related words to exclude non weather related tweets 
for tweet in latest_tweets: 
    weather_tweet = tweet.find('p').text
    if 'Sol' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 83 (2019-02-19) low -95.0ºC (-139.0ºF) high -11.7ºC (11.0ºF)
winds from the SW at 4.1 m/s (9.2 mph) gusting to 10.5 m/s (23.6 mph)
pressure at 7.20 hPa


### Mars Facts

In [9]:
#Visit Mars facts url 
mf_url = 'http://space-facts.com/mars/'

# Read table using pandas
mars_facts = pd.read_html(mf_url)
mars_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [10]:
#Convert result to pandas dataframe
mars_df = mars_facts[0]

#Assign columns ['Fact', 'Value']
mars_df.columns = ['Fact','Value']

#Set index
mars_df.set_index('Fact', inplace=True)

In [11]:
#Save result to a table
mars_df.to_html('mars_facts.html')

# Display mars_df
mars_df

,Value
Fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [31]:
# Visit the USGS Astrogeology site
site = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(site)

In [26]:
html_hemi = browser.html
soup = BeautifulSoup(html_hemi, "html.parser")
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup=BeautifulSoup(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [27]:
mars_hemisphere

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]